# 1. 문제 정의

## 사용 데이터

데이터 불러오기

In [25]:
import pandas as pd
df = pd.read_csv('../data/classification/vehicle.csv')
X = df.drop('y', axis = 1)
y = df['y']

## 하이퍼 파라미터 범위 및 평가방법

고려하는 모델은 LightGBM이며 하이퍼 파라미터는 결정 나무 개수(n_estimator), 학습률(learning_rate), 최대 잎 노드 개수(num_leaves) 입니다.

하이퍼 파라미터 범위
- 결정 나무 개수 : 10 ~ 100
- 학습률 : 0.001 ~ 0.5
- 최대 잎 노드 개수 : 20 ~ 120

# 2. 구성 요소 정의

## 목적 함수 정의

목적 함수는 하이퍼 파라미터에 따른 5-겹 교차 검증 방식으로 계산한 정확도 입니다.

목적 함수

In [26]:
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier as LGBC
def obj_func(parameter):
    model = LGBC(
        random_state = 2022,
        n_estimators = int(parameter[0]),
        learning_rate = parameter[1],
        num_leaves = int(parameter[2])
    )
    score = cross_val_score(model, X, y, scoring = 'accuracy', cv = 5).mean()
    return score

- 라인 3: 크기가 3인 배열 parameter를 입력받는 obj_func 함수를 정의 합니다. 여기서 0번째 요소는 결정 나무 개수, 1번째 요소는 학습률, 2번째 요소는 잎노드 개수를 나타냅니다.
- 라인 4~7: 입력 받은 하이퍼 파라미터를 갖는 LightGBM 모델 인스턴스인model을정의합니다. 이때 공정한 평가를 위해 random_state는 2022로 고정 했습니다.
- 라인 9: 5-겹교차 검증 방식으로 계산한 model의평균 정확도를 score에 저장합니다.


## 샘플러 정의
각 하이퍼 파라미터는 유니폼 분포로부터 샘플링하겠습니다. 샘플러는 다음과 같이 정의합니다.

샘플러

In [27]:
import numpy as np
from scipy.stats import uniform
def sampler(n):
    n_estimator = np.random.randint(10, 100, n)
    learning_rate = uniform(0.001, 0.5).rvs(n)
    num_leaves = np.random.randint(20, 120, n)
    parameter_arr = np.vstack([n_estimator, learning_rate, num_leaves]).T
    return parameter_arr

- 라인 4: 10부터 100 사이의 n개의 정수를 n_estimator에 저장합니다.
- 라인 5: 0.001부터 0.5 사이의 n 개의 실수를 learning_rate에 저장합니다.
- 라인 7: 크기가 n인 배열 n_estimator, learning_rate, num_leaves를 수직 방향으로 병합한 뒤, T를 사용해 전치시킵니다. 칼럼별로 샘플링한 결과를 합칠 때 사용하는 테크닉입니다.

## 칼럼별 샘플링 결과 병합 테크닉

## 획득 함수 정의
획득 함수로 개선 기대 함수를 사용하겠습니다.

획득 함수

In [28]:
from scipy.stats import norm
def EI(X_new, model, best_mu, e = 0.01):
    mu, sigma = model.predict(X_new, return_std = True)
    z = np.zeros(len(X_new))
    z[sigma > 0] = ((mu - best_mu -e)/sigma)[sigma > 0]
    return (mu - best_mu - e) * norm.cdf(z) + sigma * norm.pdf(z)

# 3. 메인 코드

## 메인 함수
초기에 n1 개의 해를 평가하고, 매 이터레이션마다 n2 개의 후보 해를 샘플링해서 획득 함수를 사용해 k개의 해를 선택하여 평가합니다.

In [29]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

def main(n1, n2, k, num_iter):
    P = sampler(n1)
    E = np.apply_along_axis(obj_func, axis = 1, arr = P)
    for _ in range(num_iter):
        model = GPR(kernel = RBF() + WhiteKernel(), random_state = 2022).fit(P, E)
        best_mu = max(model.predict(P))
        P_new = sampler(n2)
        score_list = EI(P_new, model, best_mu)

        P_new_star = P_new[(-score_list.argsort())[:k]]
        E_new = np.apply_along_axis(obj_func, axis = 1, arr = P_new_star)
        P = np.vstack([P, P_new_star])
        E = np.append(E, E_new)
    
    return P[E.argmax()], E.max()

- 라인 5~6: n1 개의 해를 임의로 샘플링한 결과를 P에 저장하고 obj_func 함수로 P의 각 요소를 평가한 결과를 E에 저장합니다.
- 라인 10~11: n2개의 해를 임의로 샘플링하여 획득 함수인 EI로 각각의 해를 평가 합니다.
- 라인 13: P_new에서 EI함수의 출력이 가장 큰 상위 k 개의 해를 P_new_star에 저장합니다.
- 라인 14: P_new_star에 있는 해를 obj_func 함수로 평가하고 그 결과를 E_new에 저장합니다.
- 라인 15~16: P_new_star와 E_new를 각각 P와 E에 추가합니다.

## 하이퍼 파라미터 튜닝 수행

In [30]:
print(main(100, 100, 10, 50))

(array([90.      ,  0.431995, 37.      ]), 0.7931569787678384)


- 라인 1: n1 = 100, n2 = 100, k = 10, num_iter = 50으로 설정했습니다.